In [28]:
files=16
baseFileName=r"D:\Users\Students\Daniele\records\Physic\Physic"
toWrite=r"D:\Users\Students\Daniele\records\OneCar_db.txt"

def Name(base,i):
    final=base
    if(i>0):
        final +=" - Copia"
    if(i>1):
        final +=f" ({i})"
    final+=".txt"
    return final

def ReadAndWrite(read,write,initial_dash):
    with open(read) as f:
        lines=f.readlines()
        lines=lines[1:-1] #la prima linea è sempre '\n' l'ultima potrebbe avere avuto problemi visto che il programma si è chiuso
        f.close()
    
    with open(write,'a') as f1:
       
        if(initial_dash):
            lines[0]='_'+lines[0]
        f1.writelines(lines)
        f1.close()

for i in range(files):
    file=Name(baseFileName,i)
    ReadAndWrite(file,toWrite,(i!=0))

    

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


SEQUENCE_LENGTH=20
ONLY_ONE_CAR=True
CARS=4
FEATURES=5
DISCARD=2
COLUMNS=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME"]

def normalize_df(df,minimum,maximum):
    normalized=(df-minimum)/(maximum-minimum)
    normalized["RACE"]=df["RACE"]
    return normalized

      
    
def single_care_dataframe(path):
    df_gara=pd.read_csv(path, sep=";", header=None , decimal=',',names=COLUMNS)
    #create new column called race
    df_gara["RACE"]=0
    # fill race column based on the cumulative sum of rows starting with '_'
    #idx_gara stores each row wich starts with '_'
    idx_gara=(df_gara[df_gara["Player"].str.startswith("_")].index)
    df_gara.loc[idx_gara,"Player"]=df_gara.loc[idx_gara,"Player"].str.replace("_","")#replace name without the _

    df_gara.loc[idx_gara,"RACE"]=1
    df_gara.RACE=df_gara.RACE.cumsum()
    #Create new column Length wich specifiens the total length of a race
    df_gara["LENGTH"]=df_gara.groupby("RACE")["Player"].transform("count")
    #if race is lewer then a minimum then it is discarded
    df_races=df_gara.query(f"LENGTH > {SEQUENCE_LENGTH/0.25}").reset_index(drop=True)
    df_races.drop(["Player","LENGTH"],axis=1,inplace=True)

    return df_races
    #Since
    #dfs=[]
    #for i in range(DISCARD+1):
    #    df_races["GROUP"]=i
    #    temp=df_races.iloc[i::DISCARD+1]
    #    temp.reset_index(drop=True,inplace=True)
    #    dfs.append(temp)
    #
    #return dfs

def subtraction_columns(df):
    df_copy=df.shift(1,fill_value=0)
    cols = df.columns.difference(['RACE'])
    df[cols] = df[cols].sub(df_copy[cols])
    df["ROT"]=(df["ROT"]+180)%360-180
    df.iloc[0,:-1]=0
    return df

def get_split(x,first,second):
    first =int(x.shape[0]*first)
    second = int(x.shape[0]*second) 
    return x[first:second]
    
def split_train_validation_test(df,group_col,train_split=0.5,val_split=0.25,test_split=0.25):
    val_split +=train_split
    
    if val_split >1:
        raise ValueError(
            f"Train + Validation split cannot be higher tan 1 given {val_split}"
        )
     
    #group by race (and player name for single car) and create a new array containing foreach race a dataset
    df_train= df.groupby(group_col,group_keys=False).apply(get_split, first = 0, second= train_split)
    df_val= df.groupby(group_col,group_keys=False).apply(get_split, first = train_split, second= val_split)
    df_test= df.groupby(group_col,group_keys=False).apply(get_split, first = val_split, second= 1)
    
    
    #since each race was plittend into train,val and test the result of previous operation is an array containing the data 
    #foreach race, therefore to have the end dataframe we must concatenate each element
    df_train=recreate_dataframe(df_train)
    df_val=recreate_dataframe(df_val)
    df_test=recreate_dataframe(df_test)
    
    return df_train, df_val,df_test

def recreate_dataframe(series):
    columns=["DIFF_X","DIFF_Z","DIFF_VEL_X","DIFF_VEL_Z","DIFF_ROT","TIME","RACE","GROUP"] 
    series.columns=columns
    df=series.reset_index(drop=True)
    #v#alues=series.values
    #df=pd.DataFrame(values[0],columns=columns)
    #
#
    #for serie in values[1:]:
    #    df= pd.concat([df, pd.DataFrame(serie,columns=columns)],ignore_index=True)
    return df

def batch_generator(df):
    
    #crea un nuovo dataframe con sequence_length elementi per un numero di volte pari al batch
    dropped_df=df.drop(["TIME","RACE","GROUP"],axis=1).reset_index(drop=True)
    for i in range(len(dropped_df)-SEQUENCE_LENGTH):
        inputs=np.array(dropped_df.loc[i:SEQUENCE_LENGTH-1+i,:].values)
        targets=dropped_df.loc[SEQUENCE_LENGTH+i,:].values
        yield inputs,targets
  
def Generator(df):
    grouped=df.groupby(["RACE","GROUP"],group_keys=False).apply(batch_generator)
    for group in grouped:
        for single in group:
            yield single
            
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,batch_size,df,max_batch):
        self.batch_size=batch_size
        self.df=df
        self.df_length=len(df.index)-(df["RACE"].nunique()*SEQUENCE_LENGTH*CARS)
        self.max_batch=max_batch            
        print(f'Length: {len(df.index)} races: {df["RACE"].nunique()} n batches: {self.df_length} / {batch_size}')
        self.on_epoch_end()
        #self.generator=generator_function(sequence_length,path,totFiles)
        
    def __getitem__(self,index):
        X=[]
        Y=[]
        for i in range(self.batch_size):
            #while True:
            #    x,y=next(self.generator)
            #    x_shape=np.shape(x)
            #    if x_shape[0]==x_shape[1]:
            #        break
            x,y=next(self.generator)
            X.append(x)
            Y.append(y)
            
        #print(np.shape(X))    
        tensor_x=tf.constant(X)
        tensor_y=tf.constant(Y)
        return tensor_x,tensor_y
    
    def __len__(self):
        value=int(self.df_length/self.batch_size)-1
        if value>self.max_batch:
            value=self.max_batch
        return value
    
    def on_epoch_end(self):
        self.generator=Generator(self.df)

In [ ]:
df=single_care_dataframe(toWrite)

In [ ]:
df.head()

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(subtraction_columns)

#dfs=[]
#for i in range(len(df)):
#    dfs.append(SubtractDF(df[i]))
#
#united=dfs[0]
#for i in range(1,len(df)):
#    united=pd.concat([united,dfs[i] ],ignore_index=True)
d=SubtractDF(df)

In [ ]:
d.min()

In [ ]:
d.max()

In [ ]:
id=d["VEL_X"].idxmax()
df.loc[id-2:id+2]

In [ ]:
def checkNaN(df):
    df.isnull().sum()
    return df[df.isnull().T.any()]
checkNaN(df_train)


In [ ]:
checkNaN(df)

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [18]:
path=r"D:\Users\Students\Daniele\records\Physic\Physic - Copia (9).txt"
columns=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME"]

In [19]:
df=single_care_dataframe(path)

In [20]:
df.tail()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE
41704,26.37488,173.8367,-6.738967,28.39268,344.6662,-0.331560,-12.701250,0.135517,16,5,-0.048335,0.335645,834.08,1
41705,26.23535,174.4045,-6.978809,28.39773,344.3858,-0.244739,-11.992070,0.252724,16,5,-0.050195,0.350788,834.10,1
41706,26.08838,174.9700,-7.344018,28.26927,344.0152,-0.323391,-18.260460,-6.423378,16,5,-0.052155,0.365866,834.12,1
41707,25.93433,175.5327,-7.701542,28.13634,343.5414,-0.413422,-17.876200,-6.646538,16,5,-0.054209,0.380872,834.14,1
41708,25.77771,176.0950,-7.832692,28.10911,343.1876,-0.308802,-6.557512,-1.361465,16,5,-0.056297,0.395866,834.16,1


In [8]:
delta=subtraction_columns(df)

In [9]:
delta.tail()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE
3938,-0.50500,0.3218,0.23052,0.22497,0.6193,0.119980,5.826094,-3.033110,0,0,-0.006733,0.004290,0.01999,0
3939,-0.50122,0.3274,0.19476,0.27815,0.7419,0.106935,-1.788046,2.659080,0,0,-0.006683,0.004365,0.02001,0
3940,-0.49610,0.3329,0.25548,0.27474,0.8654,0.107837,3.035926,-0.170420,0,0,-0.006615,0.004438,0.02000,0
3941,-0.49048,0.3389,0.27427,0.29854,0.9693,0.090657,0.939180,1.190090,0,0,-0.006540,0.004518,0.02000,0
3942,-0.48974,0.3400,0.04226,0.06514,0.7320,-0.206894,-11.599732,-11.670016,0,0,-0.006530,0.004534,0.02000,0


In [10]:
delta.max()

X              0.623200
Z              0.721920
VEL_X          0.933710
VEL_Z          1.238540
ROT            1.940100
ANG_VEL_Y      0.701146
ACC_X         66.354060
ACC_Z         49.761250
TILE          18.000000
TILE_IND       1.000000
X_RELATIVE     0.993578
Z_RELATIVE     0.994684
TIME           0.020010
RACE           0.000000
dtype: float64

In [22]:
df.max()

X             465.533200
Z             350.414300
VEL_X          31.366710
VEL_Z          34.566710
ROT           359.972400
ANG_VEL_Y       1.980852
ACC_X         130.815200
ACC_Z          59.615610
TILE           19.000000
TILE_IND       32.000000
X_RELATIVE      0.499997
Z_RELATIVE      0.627376
TIME          834.160000
RACE            1.000000
dtype: float64

In [23]:
ids=df["ACC_X"].idxmax()
df[ids-2:ids+2]

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE
34276,-64.41040,6.989746,-2.654520,26.353750,354.6909,-0.042258,2.998197,4.101563,16,0,0.241195,-0.113607,685.52,0
34277,-64.46265,7.518555,-2.616303,26.434740,354.6544,-0.031846,1.910818,4.049778,16,0,0.240498,-0.099505,685.54,0
34278,-91.80000,-15.800000,0.000000,0.000000,0.0000,-0.024160,130.815200,-1321.737000,4,32,-0.124001,0.389333,685.56,1
34279,-91.80005,-15.799320,-0.003261,0.034673,359.9724,-0.024136,-0.163063,1.733668,4,32,-0.124001,0.389342,685.58,1


In [17]:
def checkNaN(df):
    print(df.isnull().sum())
    return df[df.isnull().T.any()]
checkNaN(df)

X             0
Z             0
VEL_X         0
VEL_Z         0
ROT           0
ANG_VEL_Y     0
ACC_X         0
ACC_Z         0
TILE          0
TILE_IND      0
X_RELATIVE    0
Z_RELATIVE    0
TIME          0
RACE          0
dtype: int64


,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE


In [24]:
#prova 1
#vel - rot- tile-tile ind- relative